In [1]:
import pandas as pd
import numpy as np
import xarray as xr

import os
import sys
from tqdm.auto import tqdm

import matplotlib.pyplot as plt
import seaborn as sns

from src.helpers import *
from src.visualize import *
from src.trainer import *
from Models.models import *
from Models.SimpleRNN_NC import SimpleRNN_NC

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.optim import lr_scheduler
from sklearn.metrics import *
from copy import deepcopy
import torch.utils.data as data
from torch.utils.data import Dataset

import pickle
import math

from hypnettorch.hnets import HyperNetInterface

/home/nerea/anaconda3/envs/sinthlab/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Helper function to convert between numpy arrays and tensors
to_t = lambda array: torch.tensor(array, device='cpu', dtype=dtype)  #device
to_t_eval =  lambda array: torch.tensor(array, device='cuda', dtype=dtype)  #device
from_t = lambda tensor: tensor.to("cpu").detach().numpy()

## Load data

In [3]:
name = 'Chewie'
date = '1007'
fold = 4
target_variable = 'vel'

In [4]:
#@title Helper functions for plotting (run this cell!)
sns.set_context("notebook")

# initialize a color palette for plotting
palette = sns.xkcd_palette(["windows blue",
                            "red",
                            "medium green",
                            "dusty purple",
                            "orange",
                            "amber",
                            "clay",
                            "pink",
                            "greyish"])

In [5]:
to_t_eval =  lambda array: torch.tensor(array, device=device, dtype=dtype)  

## Load pre-processed data

In [6]:
data_path = './Data/Processed_Data/Tidy_'+name+'_'+date+'.pkl'

with open(data_path, 'rb') as file:
    tidy_df = pickle.load(file)

In [7]:
baseline_df = tidy_df.loc[tidy_df['epoch'] == 'BL']

In [8]:
force_df =  tidy_df.loc[tidy_df['epoch'] == 'AD']

We need to consider only the trials for which the monkey has already adapted to the perturbation.

In [9]:
ids_to_keep = force_df.id.unique()[50:]

The baseline subset has a total of 170 trials, whereas the perturbation one contains 201 trials, we can for now try to remove the first 50 trials from the perturbation subset.

In [10]:
force_df = force_df.loc[force_df.id.isin(ids_to_keep)]

## Get train-val-test split

In [11]:
xx_train_base, yy_train_base, xx_val_base, yy_val_base,\
      xx_test_base, yy_test_base, info_train_base, info_val_base,\
          info_test_base, list_mins_base, \
            list_maxs_base= get_dataset(baseline_df, fold, target_variable= target_variable, no_outliers = False, force_data = True)

Train trials 109
Test trials  34
Val trials 27
We are testing the optimization method on fold  4


In [12]:
xx_train_force, yy_train_force, xx_val_force, yy_val_force,\
      xx_test_force, yy_test_force, info_train_force, info_val_force,\
          info_test_force,  list_mins_force, \
            list_maxs_force = get_dataset(force_df, fold, target_variable= target_variable, no_outliers = False, force_data = True)

Train trials 97
Test trials  30
Val trials 24


We are testing the optimization method on fold  4


In [13]:
xx_train_all, yy_train_all, xx_val_all, yy_val_all, \
    xx_test_all, yy_test_all, info_train_all, \
    info_val_all, info_test_all,  list_mins_all,\
          list_maxs_all = get_dataset(tidy_df,fold, target_variable= target_variable, no_outliers = False
                                      , force_data = True)

Train trials 211
Test trials  66
Val trials 53
We are testing the optimization method on fold  4


In [14]:
# Specify that we want our tensors on the GPU and in float32
device = torch.device('cuda:0') #suposed to be cuda
#device = torch.device('cpu') 
dtype = torch.float32
path_to_models = './Models/Models_Force'

# Set the seed for reproducibility
seed_value = 42
torch.manual_seed(seed_value)
torch.cuda.manual_seed(seed_value)  # If using CUDA

num_dim_output = yy_train_base.shape[2]
num_features = xx_train_base.shape[2]

In [15]:
def weight_reset(m):
    reset_parameters = getattr(m, "reset_parameters", None)
    if callable(reset_parameters):
        m.reset_parameters()

## Model

In [16]:
def create_state_dict(param_names, param_values):
    s_d = {}
    for n,v in zip(param_names, param_values):
        s_d[n] = v
    return s_d

In [17]:
import torch
import torch.nn as nn

class RNN_Main_Model(nn.Module):
    def __init__(self, hnet_output, 
                 num_features = 124, 
                hidden_size= 3, 
                num_layers = 2, 
                out_dims = 6,
                dropout = 0.5,
                bias = True,
                LSTM_ = False):
        
        super(RNN_Main_Model, self).__init__()
        self.num_features = num_features
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.hnet_output = hnet_output
        self.bias = bias
        self.out_features = out_dims
        self.LSTM_ = LSTM_

        self.dropout = nn.Dropout(p= dropout) #trial.suggest_float('dropout_1', 0.1, 0.9)

        # Define recurrent layer
        self.rnn = nn.RNN(self.num_features, self.hidden_size, self.num_layers, self.bias, batch_first = True, bidirectional = False)
        names_p = [name for name, _ in self.rnn.named_parameters()]
        self.hnet_output_dict = create_state_dict(names_p,hnet_output[2:] )

        # Define recurrent layer (LSTM)
        if self.LSTM_:
            self.rnn = nn.LSTM(self.num_features, self.hidden_size, self.num_layers, self.bias, batch_first = True, bidirectional = False)
            names_p = [name for name, _ in self.rnn.named_parameters()]
            self.hnet_output_dict = create_state_dict(names_p,hnet_output[2:] )      

        self.selu = nn.SELU()      

    def forward(self, x, hx=None):
        # Forward pass
        if hx is None:
            if self.LSTM_:
                h0 = torch.randn(self.num_layers, x.size(0), self.hidden_size, device=x.device) * 0.1
                c0 = torch.randn(self.num_layers, x.size(0), self.hidden_size, device=x.device) *0.1 # Initialize cell state
                hx = (h0, c0)
            else:
                hx = torch.randn(self.num_layers, x.size(0), self.hidden_size, device=x.device) * 0.1
        
        # Perform RNN operation
        x, _  = torch.func.functional_call(self.rnn, self.hnet_output_dict, (x, hx))
        x = self.dropout(x)
        x = self.selu(x) 
        output =  F.linear(x, self.hnet_output[0], bias=self.hnet_output[1])
        
        return output.squeeze() 


#### Define a template model only to get automatically the model parameters shapes

In [18]:
num_dim_output = yy_train_base.shape[2]
num_features = xx_train_base.shape[2]

# Hyperparameters LSTM class (from force model without hnet)
# Define hyperparameters

#Hyperparameters objective and regularization
alpha_reg = 1e-5
l1_ratio_reg = 0.5

lr = 0.00001
loss_function = huber_loss
delta = 8  # hyperparameter for huber loss

# Hyperparameters LSTM class
n_hidden_units = 300
num_layers = 1
input_size = 49
dropout = 0.2

#Other training hyperparameters

lr_gamma= 1.37 #for scheduler
lr_step_size = 10 #for scheduler

seq_length_LSTM= 19
batch_size_train= 25
batch_size_val = 25

torch.manual_seed(42)


In [19]:
template_m = Causal_Simple_RNN(num_features=num_features, 
                    hidden_units= n_hidden_units, 
                    num_layers = num_layers, 
                    out_dims = num_dim_output, ).to(device)

In [20]:
param_shapes = [p.shape for p in list(template_m.parameters())]

In [21]:
param_shapes

[torch.Size([2, 300]),
 torch.Size([2]),
 torch.Size([300, 130]),
 torch.Size([300, 300]),
 torch.Size([300]),
 torch.Size([300])]

In [22]:
from hypnettorch.hnets import HMLP

num_conditions = 2
size_task_embedding = 8

hnet = HMLP(param_shapes, uncond_in_size=0,
             cond_in_size=size_task_embedding,
            layers=[13], 
            num_cond_embs=num_conditions).to(device)

Created MLP Hypernet.
Hypernetwork with 1822961 weights and 130202 outputs (compression ratio: 14.00).
The network consists of 1822945 unconditional weights (1822945 internally maintained) and 16 conditional weights (16 internally maintained).


In [23]:
for param in hnet.parameters():
    param.requires_grad = True

In [24]:
w_test = hnet(cond_id = 0)

In [25]:
LSTM_ = False

In [26]:
model = RNN_Main_Model(num_features= num_features, hnet_output = w_test,  hidden_size = n_hidden_units,
                            num_layers= num_layers,out_dims=num_dim_output,  
                            dropout= dropout,  LSTM_ = LSTM_).to(device)

In [27]:
for param in model.parameters():
    param.requires_grad = False

In [28]:
def init_hnet(model):
    # Initialize weights
    for i, layer in enumerate(model.layer_weight_tensors[:-1]):
        nn.init.kaiming_uniform_(layer, mode='fan_in', nonlinearity='relu')
        if model.has_bias:
            nn.init.zeros_(model.layer_bias_vectors[i])
        

In [29]:
def reg_hnet(weights, alpha, l1_ratio):
    
    """
    Implement an L1-L2 penalty on the norm of the model weights.

    model: MLP
    alpha: scaling parameter for the regularization.
    l1_ratio: mixing parameter between L1 and L2 loss.

    Returns:
    reg: regularization term
    """
    l1_loss = 0
    l2_loss = 0

    # Accumulate L1 and L2 losses for weight matrices in the model
    

    weights_ =  [i for i in weights if len(i.shape)==2]

    for weight_tensor in weights_[:2]:
        l1_loss += torch.sum(torch.abs(weight_tensor))
        l2_loss += torch.sum(weight_tensor.pow(2))

    reg = l1_ratio * l1_loss + (1 - l1_ratio) * l2_loss

    reg = alpha * reg

    return reg

In [30]:
# Anirudh's Function

def get_current_targets(task_id, hnet):
    r"""For all :math:`j < \text{task\_id}`, compute the output of the
    hypernetwork. This output will be detached from the graph before being added
    to the return list of this function.

    Note, if these targets don't change during training, it would be more memory
    efficient to store the weights :math:`\theta^*` of the hypernetwork (which
    is a fixed amount of memory compared to the variable number of tasks).
    Though, it is more computationally expensive to recompute
    :math:`h(c_j, \theta^*)` for all :math:`j < \text{task\_id}` everytime the
    target is needed.

    Note, this function sets the hypernet temporarily in eval mode. No gradients
    are computed.

    See argument ``targets`` of :func:`calc_fix_target_reg` for a use-case of
    this function.

    Args:
        task_id (int): The ID of the current task.
        hnet: An instance of the hypernetwork before learning a new task
            (i.e., the hypernetwork has the weights :math:`\theta^*` necessary
            to compute the targets).

    Returns:
        An empty list, if ``task_id`` is ``0``. Otherwise, a list of
        ``task_id-1`` targets. These targets can be passed to the function
        :func:`calc_fix_target_reg` while training on the new task.
    """
    # We temporarily switch to eval mode for target computation (e.g., to get
    # rid of training stochasticities such as dropout).
    hnet_mode = hnet.training
    hnet.eval()

    ret = []

    with torch.no_grad():
        W = hnet.forward(cond_id=list(range(task_id)), ret_format="sequential")
        ret = [[p.detach() for p in W_tid] for W_tid in W]

    hnet.train(mode=hnet_mode)

    return ret


In [31]:
def calc_fix_target_reg(
    hnet,
    task_id,
    targets=None,
    dTheta=None,
    dTembs=None,
    mnet=None,
    prev_theta=None,
    prev_task_embs=None,
    batch_size=None,
    reg_scaling=None,
):
    r"""This regularizer simply restricts the output-mapping for previous
    task embeddings. I.e., for all :math:`j < \text{task\_id}` minimize:

    .. math::
        \lVert \text{target}_j - h(c_j, \theta + \Delta\theta) \rVert^2

    where :math:`c_j` is the current task embedding for task :math:`j` (and we
    assumed that ``dTheta`` was passed).

    Args:
        hnet: The hypernetwork whose output should be regularized; has to
            implement the interface
            :class:`hnets.hnet_interface.HyperNetInterface`.
        task_id (int): The ID of the current task (the one that is used to
            compute ``dTheta``).
        targets (list): A list of outputs of the hypernetwork. Each list entry
            must have the output shape as returned by the
            :meth:`hnets.hnet_interface.HyperNetInterface.forward` method of the
            ``hnet``. Note, this function doesn't detach targets. If desired,
            that should be done before calling this function.

            Also see :func:`get_current_targets`.
        dTheta (list, optional): The current direction of weight change for the
            internal (unconditional) weights of the hypernetwork evaluated on
            the task-specific loss, i.e., the weight change that would be
            applied to the unconditional parameters :math:`\theta`. This
            regularizer aims to modify this direction, such that the hypernet
            output for embeddings of previous tasks remains unaffected.
            Note, this function does not detach ``dTheta``. It is up to the
            user to decide whether dTheta should be a constant vector or
            might depend on parameters of the hypernet.

            Also see :func:`utils.optim_step.calc_delta_theta`.
        dTembs (list, optional): The current direction of weight change for the
            task embeddings of all tasks that have been learned already.
            See ``dTheta`` for details.
        mnet: Instance of the main network. Has to be provided if
            ``inds_of_out_heads`` are specified.
        prev_theta (list, optional): If given, ``prev_task_embs`` but not
            ``targets`` has to be specified. ``prev_theta`` is expected to be
            the internal unconditional weights :math:`theta` prior to learning
            the current task. Hence, it can be used to compute the targets on
            the fly (which is more memory efficient (constant memory), but more
            computationally demanding).
            The computed targets will be detached from the computational graph.
            Independent of the current hypernet mode, the targets are computed
            in ``eval`` mode.
        prev_task_embs (list, optional): If given, ``prev_theta`` but not
            ``targets`` has to be specified. ``prev_task_embs`` are the task
            embeddings (conditional parameters) of the hypernetwork.
            See docstring of ``prev_theta`` for more details.
        batch_size (int, optional): If specified, only a random subset of
            previous tasks is regularized. If the given number is bigger than
            the number of previous tasks, all previous tasks are regularized.

            Note:
                A ``batch_size`` smaller or equal to zero will be ignored
                rather than throwing an error.
        reg_scaling (list, optional): If specified, the regulariation terms for
            the different tasks are scaled arcording to the entries of this
            list.

    Returns:
        The value of the regularizer.
    """
    assert isinstance(hnet, HyperNetInterface)
    assert task_id > 0
    # FIXME We currently assume the hypernet has all parameters internally.
    # Alternatively, we could allow the parameters to be passed to us, that we
    # will then pass to the forward method.
    assert hnet.unconditional_params is not None and len(hnet.unconditional_params) > 0
    assert targets is None or len(targets) == task_id
    assert targets is None or (prev_theta is None and prev_task_embs is None)
    assert prev_theta is None or prev_task_embs is not None
    # assert prev_task_embs is None or len(prev_task_embs) >= task_id
    assert dTembs is None or len(dTembs) >= task_id
    assert reg_scaling is None or len(reg_scaling) >= task_id

    # Number of tasks to be regularized.
    num_regs = task_id
    ids_to_reg = list(range(num_regs))
    if batch_size is not None and batch_size > 0:
        if num_regs > batch_size:
            ids_to_reg = np.random.choice(
                num_regs, size=batch_size, replace=False
            ).tolist()
            num_regs = batch_size

    # FIXME Assuming all unconditional parameters are internal.
    assert len(hnet.unconditional_params) == len(hnet.unconditional_param_shapes)

    weights = dict()
    uncond_params = hnet.unconditional_params
    if dTheta is not None:
        uncond_params = hnet.add_to_uncond_params(dTheta, params=uncond_params)
    weights["uncond_weights"] = uncond_params

    if dTembs is not None:
        # FIXME That's a very unintutive solution for the user.
        assert (
            hnet.conditional_params is not None
            and len(hnet.conditional_params) == len(hnet.conditional_param_shapes)
            and len(hnet.conditional_params) == len(dTembs)
        )
        weights["cond_weights"] = hnet.add_to_uncond_params(
            dTembs, params=hnet.conditional_params
        )

    if targets is None:
        prev_weights = dict()
        prev_weights["uncond_weights"] = prev_theta
        # FIXME We just assume that `prev_task_embs` are all conditional
        # weights.
        prev_weights["cond_weights"] = prev_task_embs

    reg = 0

    for i in ids_to_reg:
        weights_predicted = hnet.forward(cond_id=i, weights=weights)

        if targets is not None:
            target = targets[i]
        else:
            # Compute targets in eval mode!
            hnet_mode = hnet.training
            hnet.eval()

            # Compute target on the fly using previous hnet.
            with torch.no_grad():
                target = hnet.forward(cond_id=i, weights=prev_weights)
            target = [d.detach().clone() for d in target]

            hnet.train(mode=hnet_mode)

        # Regularize all weights of the main network.
        W_target = torch.cat([w.view(-1) for w in target])
        W_predicted = torch.cat([w.view(-1) for w in weights_predicted])

        reg_i = (W_target - W_predicted).pow(2).sum()

        if reg_scaling is not None:
            reg += reg_scaling[i] * reg_i
        else:
            reg += reg_i

    return reg / num_regs

In [32]:
def train_current_task(
        model, 
        hnet,
        y_train, 
        x_train,
        y_val,
        x_val,
        optimizer,
        scheduler,
        calc_reg = False,
        cond_id = 0,
        lr=0.0001,
        lr_step_size=10,
        lr_gamma=0.9,
        sequence_length_LSTM=15,
        batch_size_train = 25,
        batch_size_val = 25,
        num_epochs=1000, 
        delta = 8,      
        beta=0,           
        regularizer=None,
        l1_ratio = 0.5,
        alpha_A = 1e-5,  
        alpha_B = 1e-5,        
        early_stop = 10,
        LSTM_ = LSTM_,
        chunks = False):
    
    # Compute weights that result from hnet from all previous tasks
    if calc_reg == True:
        reg_targets = get_current_targets(cond_id, hnet)
        prev_hnet_theta = None
        prev_task_embs = None
    
    
    
    # Keep track of the best model's parameters and loss
    best_model_wts = deepcopy(model.state_dict())
    best_loss = 1e8

    # Enable anomaly detection for debugging
    torch.autograd.set_detect_anomaly(True)

    # Track the train and validation loss
    train_losses = []
    val_losses = []
    # Counters for early stopping
    not_increased = 0
    end_train = 0
    
    # Reshape data for the LSTM
    train_dataset = SequenceDataset(y_train, x_train, sequence_length=sequence_length_LSTM)
    val_dataset = SequenceDataset(y_val, x_val, sequence_length=sequence_length_LSTM)
    loader_train = data.DataLoader(train_dataset, batch_size=batch_size_train, shuffle=True)
    loader_val = data.DataLoader(val_dataset, batch_size=batch_size_val, shuffle=True)

    # Initialize h0 and c0 outside the model
    if LSTM_ == True:

        h0 = torch.randn(num_layers, batch_size_train, n_hidden_units, device=device) * 0.1
        c0 = torch.randn(num_layers, batch_size_train, n_hidden_units, device=device) *0.1 # Initialize cell state
        hx = (h0, c0) 
    else:
        hx = torch.randn(num_layers, batch_size_train, n_hidden_units, device=device) * 0.1
    # Loop through epochs
    for epoch in np.arange(num_epochs):
        for phase in ['train', 'val']:
            # set model to train/validation as appropriate
            if phase == 'train':
                model.train()
                loader = loader_train
            else:
                model.eval()
                loader = loader_val

            # Initialize variables to track loss and batch size
            running_loss = 0
            running_size = 0        

            # Iterate over batches in the loader
            for data in loader:

                # Define data for this batch
                x = data[0].to('cuda')
                y = data[1].to('cuda')

                if phase == "train":
                    with torch.set_grad_enabled(True):
                        optimizer.zero_grad()

                        # Forward pass through both models
                        W = hnet(cond_id=0)
                        model = RNN_Main_Model(
                            num_features= num_features, 
                            hnet_output = W,  
                            hidden_size = n_hidden_units,
                            num_layers= num_layers, 
                            out_dims=num_dim_output,  
                            dropout= dropout, 
                            LSTM_ = LSTM_).to(device)
                                    
                        y_pred = model(x, hx)
                        
                        # Compute loss from the current task
                        loss_task = huber_loss(y_pred, y, delta = delta)
                        
                        # Add regularization from the previous tasks
                        if calc_reg:
                            loss_reg = calc_fix_target_reg(
                                hnet,
                                cond_id,
                                targets=reg_targets,
                                mnet=model,
                                prev_theta=prev_hnet_theta,
                                prev_task_embs=prev_task_embs,)

                            loss_t = loss_task + beta * loss_reg 
                        else:
                            loss_t = loss_task
                        
                        
                    
                        # Compute gradients and perform an optimization step
                        loss_t.backward()
                        optimizer.step()
                else:
                    # just compute the loss in validation phase
                    # Compute FIRST loss.
                    W = hnet(cond_id=0)
                    model = RNN_Main_Model(num_features= num_features, hnet_output = W,  hidden_size = n_hidden_units,
                        num_layers= num_layers, out_dims=num_dim_output,  
                        dropout= dropout, LSTM_ = LSTM_).to(device)    
                    y_pred = model(x, hx)
                    loss = huber_loss(y_pred, y, delta = delta)

                    loss_t = loss

                # Ensure the loss is finite
                assert torch.isfinite(loss_t)
                running_loss += loss_t.item()
                running_size += 1

            # compute the train/validation loss and update the best
            # model parameters if this is the lowest validation loss yet
            running_loss /= running_size
            if phase == "train":
                train_losses.append(running_loss)
            else:
                val_losses.append(running_loss)
                # Update best model parameters if validation loss improves
                if running_loss < best_loss:
                    best_loss = running_loss
                    best_w = W
                    best_model_wts = deepcopy(model.state_dict())
                    not_increased = 0
                else:
                    # Perform early stopping if validation loss doesn't improve
                    if epoch > 10:
                        not_increased += 1
                        # print('Not increased : {}/5'.format(not_increased))
                        if not_increased == early_stop:
                            print('Decrease LR')
                            for g in optimizer.param_groups:
                                g['lr'] = g['lr'] / 2
                            not_increased = 0
                            end_train += 1
                        
                        if end_train == 2:
                            model.load_state_dict(best_model_wts)
                            return np.array(train_losses), np.array(val_losses), best_w

        # Update learning rate with the scheduler
        scheduler.step()
        print("Epoch {:03} Train {:.4f} Val {:.4f}".format(epoch, train_losses[-1], val_losses[-1]))

    # load best model weights
    model.load_state_dict(best_model_wts)

    return np.array(train_losses), np.array(val_losses), best_w

In [33]:
lr = 0.001
beta = 1e-1

In [34]:
task_names = ['Baseline', 'Adaptation']
task_data = [baseline_df, force_df]

calc_reg = False
task_id = 0

train_losses  = {}
val_losses = {}
best_W = {}

for name, dataset_ in zip(task_names, task_data):
    if task_id >0:
        calc_reg = True

    # Set up the optimizer with the specified learning rate
    optimizer = torch.optim.Adam(hnet.internal_params, lr=lr)

    # Set up a learning rate scheduler
    scheduler = lr_scheduler.StepLR(optimizer, 
                                    step_size=lr_step_size, 
                                    gamma=lr_gamma)
    
    # Generate feature and target matrices
    x_train, y_train, x_val, y_val, \
    x_test, y_test, info_train, \
    info_val, info_test,  list_mins,\
          list_maxs = get_dataset(dataset_,fold,
                                    target_variable= target_variable,
                                    no_outliers = False, 
                                    force_data = True)
    
    train_losses_, val_losses_, best_w_ =train_current_task(
        model, 
        hnet,
        y_train, 
        x_train,
        y_val,
        x_val,
        optimizer,
        scheduler,
        calc_reg = calc_reg,
        cond_id = task_id,
        lr=lr,
        lr_step_size=5,
        lr_gamma= lr_gamma, #0.9
        sequence_length_LSTM = seq_length_LSTM, #15
        batch_size_train = batch_size_train, #15
        batch_size_val = batch_size_train, #15
        num_epochs=1000, 
        delta = 8,
        beta = beta,             
        regularizer=reg_hnet_noweights,
        l1_ratio = l1_ratio_reg, #0.5
        alpha_A = alpha_reg, 
        alpha_B = alpha_reg,     
        early_stop = 5,
        chunks = False)
    
    train_losses[name] = train_losses_
    val_losses[name] = val_losses_
    best_W[name] = best_w_

Train trials 109
Test trials  34
Val trials 27
We are testing the optimization method on fold  4


UnboundLocalError: local variable 'data' referenced before assignment

# got stuck here, some problem with variable "data"

In [ ]:
subsets = ['Training', 'Validation', 'Test']

data_base = [[xx_train_base, yy_train_base],
             [xx_val_base, yy_val_base],
             [xx_test_base, yy_test_base]]

data_force = [[xx_train_force, yy_train_force],
             [xx_val_force, yy_val_force],
             [xx_test_force, yy_test_force]]

In [ ]:
for index, [x,y] in enumerate(data_base):
    r2 = calc_explained_variance(x, y, W_base)
    print('Explained variance score for ', subsets[index], ' is : ', r2)


Explained variance score for  Training  is :  0.9358438849449158
Explained variance score for  Validation  is :  0.8675511181354523
Explained variance score for  Test  is :  0.8655193746089935


In [ ]:
for index, [x,y] in enumerate(data_force):
    r2 = calc_explained_variance(x, y, W_force)
    print('Explained variance score for ', subsets[index], ' is : ', r2)

Explained variance score for  Training  is :  0.9086236357688904
Explained variance score for  Validation  is :  0.8310559093952179
Explained variance score for  Test  is :  0.8234804272651672


In [ ]:
model_base_hnet = RNN_Main_Model(num_features= num_features, hnet_output = W_base,  hidden_size = n_hidden_units,
                            num_layers= num_layers, out_dims=num_dim_output,  
                            dropout= dropout, LSTM_ = LSTM_).to(device)  

In [ ]:
model_force_hnet = RNN_Main_Model(num_features= num_features, hnet_output = W_force,  hidden_size = n_hidden_units,
                            num_layers= num_layers, out_dims=num_dim_output,  
                            dropout= dropout, LSTM_ = LSTM_).to(device) 

In [ ]:
exp_base = 'RNN_hnet_'+name+'_'+date+'_Baseline'
exp_force = 'RNN_hnet_'+name+'_'+date+'_Force'
path_base = os.path.join(path_to_models,exp_base)
path_force = os.path.join(path_to_models,exp_force)
if not os.path.exists(path_base):
    os.makedirs(path_base)
if not os.path.exists(path_force):
    os.makedirs(path_force)
path_base_fold = os.path.join(path_base,'fold_{}.pth'.format(fold))
path_force_fold = os.path.join(path_force,'fold_{}.pth'.format(fold))
torch.save(model_base_hnet, path_base_fold)
torch.save(model_force_hnet, path_force_fold)